### Introducción


En AlixPartners trabajamos con diferentes problemas de negocio que requieren un análisis de los datos para extraer conclusiones. A continuación se describe el problema a resolver

“Fiestas Dulces” es una empresa que entrega sus artículos en los puntos de venta a terceros (consignación). El producto insignia de la empresa se consume mayoritariamente en las fiestas a fin de año. El problema es que los artículos que comercializa son perecederos. La práctica usual es descartar lo que no se vende.

En la actualidad la empresa goza de buenos ingresos, pero considerando el descarte de artículos el margen podría mejorar. Para maximizar la ganancia “Fiestas Dulces” necesita minimizar dicho descarte. Una forma de hacer esto es conseguir buenas predicciones sobre la venta de sus productos en cada punto de venta.

El objetivo de esta competencia es predecir la cantidad de artículos que se venderá en cada punto de venta en diciembre de 2014.

### Evaluación


La calificación de la competencia se logra en un 85% con el resultado de las predicciones y un 15% con un informe en donde los participantes expliquen qué tratamiento hicieron de los datos, y qué decisiones tomaron para el modelado. Se leerán todos los informes presentados, incluso aquellos con bajo puntaje en las predicciones. ¡A no desalentarse!

La calificación de la solución propuesta se hace con el error absoluto medio (MAE por su sigla en inglés) y se calcula como el promedio de las diferencias (en valor absoluto) entre las respuestas enviadas y las correctas:

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/3ef87b78a9af65e308cf4aa9acf6f203efbdeded">

Los participantes disponen de tres archivos que reúnen la información del caso, envios.txt, ventas.txt y pos.csv.

**envios.txt:**

    - fecha: Fecha de envío.
    - id_pos: Identificador único de Punto de Venta.
    - unidades_despachadas: Cantidad de productos enviados al POS en la fecha mencionada.
    - cantidad_envios: Cantidad de envíos realizados para la fecha y POS en cuestión.

**ventas.txt:**

    - fecha: Fecha de venta.
    - id_pos: Identificador único de Punto de Venta.
    - cantidad_envios: Cantidad de productos vendidos.
    - canal: Es la clase de tienda en la que se realizó la venta.

**pos.csv:**

    - id_pos: Identificador único de Punto de Venta.
    - competidores: Número de competidores en un radio de una milla del POS.
    - ingreso_mediana: Mediana del ingreso para el código postal en cuestión
    - ingreso_promedio: Ingreso promedio para el código postal en cuestión.
    - densidad_poblacional: Densidad poblacional para el código postal en cuestión.
    - pct_0a5: Porcentaje de la población de 0 a 5 años.
    - pct_5a9: Porcentaje de la población de 5 a 9 años.
    - pct_10a14: Porcentaje de la población de 10 a 14 años.
    - pct_15a19: Porcentaje de la población de 15 a 19 años.
    - pct_20a24: Porcentaje de la población de 20 a 24 años.
    - pct_25a29: Porcentaje de la población de 25 a 29 años.
    - pct_30a34: Porcentaje de la población de 30 a 34 años.
    - pct_35a39: Porcentaje de la población de 35 a 39 años.
    - pct_40a44: Porcentaje de la población de 40 a 44 años.
    - pct_45a49: Porcentaje de la población de 45 a 49 años.
    - pct_50a54: Porcentaje de la población de 50 a 54 años.
    - pct_55a59: Porcentaje de la población de 55 a 59 años.
    - pct_60a64: Porcentaje de la población de 60 a 64 años.
    - pct_65a69: Porcentaje de la población de 65 a 69 años.
    - pct_70a74: Porcentaje de la población de 70 a 74 años.
    - pct_75a79: Porcentaje de la población de 75 a 79 años.
    - pct_80a84: Porcentaje de la población de 80 a 84 años.
    - pct_85ainf: Porcentaje de la población mayor a 85 años.
    - pct_bachelors: Porcentaje de la población con título de bachelor.
    - pct_doctorados: Porcentaje de la población con título de doctor.
    - pct_secundario: Porcentaje de la población con título secundario.
    - pct_master: Porcentaje de la población con título de master.
    - pct_bicicleta: Porcentaje de utilización de bicicletas.
    - pct_omnibus: Porcentaje de utilización de omnibus.
    - pct_subtes: Porcentaje de utilización de subtes.
    - pct_taxi: Porcentaje de utilización de taxis.
    - pct_caminata: Porcentaje de transporte de a pie.
    - mediana_valor_hogar: Mediana de valor del hogar en la vecindad del POS.

**NOTAS:**
- En envios hay 13449 id_pos distintos
- En ventas hay 12302 id_pos distintos
- En pos hay 12934 id_pos distintos 
Por lo cual en envios existen a priori 13449 - 12934 = 515 pos sin identificar o sin informacion de pos
- En pos hay 110 id_pos nulos y 187 con valor igual a 'Not Available'

- En Ventas hay 472 id_pos que no se encontro su informacion de pos correspondiente (al cruzar con pos). Algo se debe hacer con esto ultimo:
    - Descartar esos registros para el entrenamiento (o bien separar el dataset en dos y entrenar dos modelos aparter y luego ensamblar)
    - Imputar los valores nulos con algun valor por default o media
    - Ver los id_pos nulos o 'Not Available' de pos que quizas corresponden a los de ventas que no se pudieron cruzar

- En Ventas hay 2 id_pos que no se encontro su informacion de envios correspondiente (al cruzar con envios). Esos registros se pueden descartar.
- Hay que tener en cuenta la fecha de envio con la de venta para poder hacer el cruce correspondientes de datos.
- Hip: La informacion en envio puede ser leaks. Validar esto

- En el datase de submittion hay 594 id_pos mas que en ventas. Estos son id_pos 'ciegos', es decir que no tenes historia de ventas por lo cual su prediccion se va a deber a la generalizacion de los datos obtenidos del modelo para aquellos que si poseemos historia de ventas.

- Todos los id_pos del dataset de submition se encuentran en el de pos. Para scorear se debe filtrar por solo esos.




In [1]:
13449 - 12934

515

In [2]:
import pandas as pd 

In [3]:
envios = pd.read_csv('../data/raw/envios.txt', sep='|')

In [4]:
envios.shape

(42434, 4)

In [5]:
envios.head()

,fecha,id_pos,unidades_despachadas,cantidad_envios
0,2012-08-17,5235,36,1
1,2012-08-24,4611,48,1
2,2012-08-24,4645,48,1
3,2012-08-24,4649,72,1
4,2012-08-24,4677,72,1


In [6]:
envios.id_pos.nunique()

13449

In [7]:
ventas = pd.read_csv('../data/raw/ventas.txt', sep='|')

In [8]:
ventas.head()

,fecha,id_pos,unidades,canal
0,2012-10-01,8495,4,MAXIKIOSCO
1,2012-10-10,8571,1,MAXIKIOSCO
2,2012-10-11,8915,1,MAXIKIOSCO
3,2012-10-12,8915,3,MAXIKIOSCO
4,2012-10-13,8923,1,MAXIKIOSCO


In [9]:
ventas.shape

(266436, 4)

In [10]:
ventas.id_pos.nunique()

12302

In [11]:
pos = pd.read_csv('../data/raw/pos.csv')

In [12]:
pos.head()

,id_pos,competidores,ingreso_mediana,ingreso_promedio,densidad_poblacional,pct_0a5,pct_5a9,pct_10a14,pct_15a19,pct_20a24,...,pct_bachelors,pct_doctorados,pct_secundario,pct_master,pct_bicicleta,pct_omnibus,pct_subtes,pct_taxi,pct_caminata,mediana_valor_hogar
0,-37,9.0,60769.03000,84420.06500,32715.700,5.260000,4.33000,4.245000,4.71000,5.935000,...,22.880000,2.395000,21.44000,10.60,1.935000,3.055000,26.00500,1.360000,16.475000,434685.4150
1,-37,6.0,39675.89000,50716.23000,29985.710,8.860000,8.60000,8.170000,7.58000,8.950000,...,10.610000,0.230000,25.97000,2.51,0.550000,2.910000,0.47000,0.000000,1.380000,143807.1500
2,-37,36.0,65351.97750,82575.15000,7610.355,6.460000,6.29625,6.088750,5.75875,7.031250,...,20.667500,1.328750,29.25625,8.87,0.257500,0.742500,0.17125,0.013750,2.360000,212977.4850
3,-37,25.0,62489.12167,84227.44333,5685.860,6.288333,7.19000,6.958333,6.15000,5.841667,...,25.001667,1.513333,22.80500,8.90,0.221667,1.056667,0.00500,0.266667,2.091667,219048.1067
4,-37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
pos.id_pos.nunique()

12934

In [14]:
submittion = pd.read_csv('../data/raw/ejemploRespuesta.csv')

In [15]:
submittion.columns = ['id_pos','score']

In [16]:
submittion.id_pos.nunique()

12896

In [17]:
submittion.describe()

,id_pos,score
count,12896.000000,12896.000000
mean,11910.906172,1.980847
std,9721.751513,1.414495
min,953.000000,0.000000
25%,6645.750000,1.000000
50%,10733.000000,2.000000
75%,14848.250000,3.000000
max,131072.000000,4.000000


In [18]:
submittion.id_pos.nunique() - ventas.id_pos.nunique()

594

In [19]:
submittion.dtypes

id_pos    int64
score     int64
dtype: object

In [20]:
pos.dtypes

id_pos                   object
competidores            float64
ingreso_mediana         float64
ingreso_promedio        float64
densidad_poblacional    float64
pct_0a5                 float64
pct_5a9                 float64
pct_10a14               float64
pct_15a19               float64
pct_20a24               float64
pct_25a29               float64
pct_30a34               float64
pct_35a39               float64
pct_40a44               float64
pct_45a49               float64
pct_50a54               float64
pct_55a59               float64
pct_60a64               float64
pct_65a69               float64
pct_70a74               float64
pct_75a79               float64
pct_80a84               float64
pct_85ainf              float64
pct_bachelors           float64
pct_doctorados          float64
pct_secundario          float64
pct_master              float64
pct_bicicleta           float64
pct_omnibus             float64
pct_subtes              float64
pct_taxi                float64
pct_cami

In [21]:
pos['id_pos'].isnull().sum()

110

In [22]:
pos = pos[pos['id_pos'].isnull() != True]

In [23]:
pos = pos[pos['id_pos']!='Not Available']

In [24]:
# convertimos a int para poder cruzar con informacion de venta
pos['id_pos'] = pos['id_pos'].astype(int)

In [25]:
train = pd.merge(ventas, pos, how='left', on='id_pos')

In [26]:
train.head()

,fecha,id_pos,unidades,canal,competidores,ingreso_mediana,ingreso_promedio,densidad_poblacional,pct_0a5,pct_5a9,...,pct_bachelors,pct_doctorados,pct_secundario,pct_master,pct_bicicleta,pct_omnibus,pct_subtes,pct_taxi,pct_caminata,mediana_valor_hogar
0,2012-10-01,8495,4,MAXIKIOSCO,14.0,49367.31,65176.325,7981.955,6.695,6.260,...,16.750,0.610,32.42,5.35,0.180,1.160,0.00,0.145,3.01,152962.485
1,2012-10-10,8571,1,MAXIKIOSCO,10.0,100524.20,134309.315,5403.320,4.250,5.765,...,25.215,3.675,16.68,13.85,0.075,2.915,3.18,0.005,0.87,325699.190
2,2012-10-11,8915,1,MAXIKIOSCO,2.0,38236.83,48551.870,9318.700,5.410,5.610,...,9.410,0.280,38.51,3.27,0.100,2.630,0.28,0.020,0.93,95502.900
3,2012-10-12,8915,3,MAXIKIOSCO,2.0,38236.83,48551.870,9318.700,5.410,5.610,...,9.410,0.280,38.51,3.27,0.100,2.630,0.28,0.020,0.93,95502.900
4,2012-10-13,8923,1,MAXIKIOSCO,9.0,31019.49,43710.370,4117.760,5.240,6.120,...,6.610,0.240,36.29,4.82,0.230,3.250,0.00,0.000,2.20,81577.770


In [27]:
train[train.competidores.isnull()].id_pos.nunique()

472

In [28]:
ventas.shape

(266436, 4)

In [29]:
# Nos quedamos con un unico id_pos, sin tener en cuenta la fecha
envios_tmp = envios[['id_pos']].drop_duplicates()

In [30]:
train.id_pos.nunique()

12302

In [31]:
train[train.id_pos.isin(envios.id_pos)].id_pos.nunique()

12300

In [32]:
train[train.id_pos.isin(envios.id_pos)!=True].id_pos.nunique()

2

In [33]:
train = pd.merge(train, envios_tmp, how='left', left_on='id_pos', right_on='id_pos')

In [34]:
train.shape

(266436, 36)

In [35]:
train.head()

,fecha,id_pos,unidades,canal,competidores,ingreso_mediana,ingreso_promedio,densidad_poblacional,pct_0a5,pct_5a9,...,pct_bachelors,pct_doctorados,pct_secundario,pct_master,pct_bicicleta,pct_omnibus,pct_subtes,pct_taxi,pct_caminata,mediana_valor_hogar
0,2012-10-01,8495,4,MAXIKIOSCO,14.0,49367.31,65176.325,7981.955,6.695,6.260,...,16.750,0.610,32.42,5.35,0.180,1.160,0.00,0.145,3.01,152962.485
1,2012-10-10,8571,1,MAXIKIOSCO,10.0,100524.20,134309.315,5403.320,4.250,5.765,...,25.215,3.675,16.68,13.85,0.075,2.915,3.18,0.005,0.87,325699.190
2,2012-10-11,8915,1,MAXIKIOSCO,2.0,38236.83,48551.870,9318.700,5.410,5.610,...,9.410,0.280,38.51,3.27,0.100,2.630,0.28,0.020,0.93,95502.900
3,2012-10-12,8915,3,MAXIKIOSCO,2.0,38236.83,48551.870,9318.700,5.410,5.610,...,9.410,0.280,38.51,3.27,0.100,2.630,0.28,0.020,0.93,95502.900
4,2012-10-13,8923,1,MAXIKIOSCO,9.0,31019.49,43710.370,4117.760,5.240,6.120,...,6.610,0.240,36.29,4.82,0.230,3.250,0.00,0.000,2.20,81577.770


In [36]:
submittion.id_pos.nunique()

12896

In [37]:
submittion.dtypes

id_pos    int64
score     int64
dtype: object

In [38]:
submittion[submittion.id_pos.isin(pos.id_pos)]['id_pos'].nunique()

12896

In [42]:
from datetime import date

In [51]:
ventas.head(20)

,fecha,id_pos,unidades,canal,fecha_norm,unidades_one_periode_before
239593,2013-12-24,63432,2,ALMACEN,2013-12-01,NaN
222039,2013-12-22,63432,1,ALMACEN,2013-12-01,2.0
205735,2013-12-20,63432,2,ALMACEN,2013-12-01,1.0
190086,2013-12-18,63432,1,ALMACEN,2013-12-01,2.0
185903,2013-12-17,63432,2,ALMACEN,2013-12-01,1.0
176768,2013-12-16,63432,1,ALMACEN,2013-12-01,2.0
173503,2013-12-15,63432,2,ALMACEN,2013-12-01,1.0
111817,2013-11-21,63432,1,ALMACEN,2013-11-01,NaN
95013,2012-12-23,63432,1,ALMACEN,2012-12-01,NaN
92333,2012-12-22,63432,1,ALMACEN,2012-12-01,1.0


In [40]:
ventas['fecha'] = pd.to_datetime(ventas['fecha'])

In [43]:
ventas['fecha_norm'] = ventas['fecha'].apply(lambda x : date(x.year,x.month,1))

In [52]:
ventas_aggr = ventas.groupby(['id_pos','fecha_norm']).unidades.sum().reset_index()

In [54]:
ventas_aggr.head(10)

,id_pos,fecha_norm,unidades
0,3142,2012-12-01,12
1,3142,2013-12-01,7
2,3143,2013-11-01,2
3,3143,2013-12-01,47
4,3144,2012-11-01,6
5,3144,2012-12-01,41
6,3144,2013-11-01,4
7,3144,2013-12-01,71
8,3146,2012-12-01,7
9,3146,2013-12-01,7


In [47]:
ventas.sort_values(['id_pos','fecha'], ascending=False, inplace=True)

In [49]:
ventas['unidades_one_periode_before'] = ventas.groupby(['id_pos','fecha_norm'])['unidades'].shift()